In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
# 指定當前的工作資料夾
import os
# 此處為google drive中的檔案路徑,drive為之前指定的工作根目錄，要加上
os.chdir("drive/Colab Notebooks")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from pandas import DataFrame

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer  # https://kknews.cc/zh-tw/other/g5qoogm.html


# kaggle 分數計算公式
def AVM(y_test, y_pred):
      y_test, y_pred = np.array(y_test), np.array(y_pred)
      Z = np.where(abs((y_pred - y_test) / y_test) <= 0.1, 1, 0)
      hit_rate = Z.sum() / len(Z)
      MAPE = (abs((y_pred - y_test) / y_test)).sum() / len(Z)
      score = hit_rate * 10000 + (1 - MAPE)
      return print(score)


# 變數轉換


# 輸入資料
data = pd.read_csv("train_2.csv", encoding='utf-8')
data_t = pd.read_csv("test.csv", encoding='utf-8')
# print(data.dtypes) # 看變數型態

# 資料清洗
# 以city分類，拿village_income_median做平均
grouped = data['village_income_median'].groupby(data["city"])
list1 = [3, 5, 6, 7, 9, 10, 12, 13, 14, 17, 21]
list2 = [597.345633, 588.768020, 568.731400, 646.278368, 1035.359111, 686.269449, 635.886657, 770.654135, 642.978935,
         630.751546, 647.720324]
for i in range(11):
      data[data["city"] == list1[i]] = data[data["city"] == list1[i]].fillna(list2[i])
for i in range(11):
      data_t[data_t["city"] == list1[i]] = data_t[data_t["city"] == list1[i]].fillna(list2[i])

# 轉換數值型態(區分類別,數值)
data[[
      "building_id", "city", "building_type", "building_use", "parking_way", "total_floor", "town",
      "I_index_50", "I_index_500", "I_index_1000", "I_index_5000", "I_index_10000", "II_index_50", "II_index_500",
      "II_index_1000", "II_index_5000", "II_index_10000", "III_index_50", "III_index_500", "III_index_1000",
      "III_index_5000", "III_index_10000", "IV_index_50", "IV_index_500", "IV_index_1000", "IV_index_5000",
      "IV_index_10000", "V_index_50", "V_index_500", "V_index_1000", "V_index_5000", "V_index_10000", "VI_index_50",
      "VI_index_500", "VI_index_1000", "VI_index_5000", "VI_index_10000", "VII_index_50", "VII_index_500",
      "VII_index_1000", "VII_index_5000", "VII_index_10000", "VIII_index_50", "VIII_index_500", "VIII_index_1000",
      "VIII_index_5000", "VIII_index_10000", "IX_index_50", "IX_index_500", "IX_index_1000", "IX_index_5000",
      "IX_index_10000", "X_index_50", "X_index_500", "X_index_1000", "X_index_5000", "X_index_10000", "XI_index_50",
      "XI_index_500", "XI_index_1000", "XI_index_5000", "XI_index_10000", "XII_index_50", "XII_index_500",
      "XII_index_1000", "XII_index_5000", "XII_index_10000", "XIII_index_50", "XIII_index_500", "XIII_index_1000",
      "XIII_index_5000", "XIII_index_10000", "XIV_index_50", "XIV_index_500", "XIV_index_1000", "XIV_index_5000",
      "XIV_index_10000"
]] = data[[
      "building_id", "city", "building_type", "building_use", "parking_way", "total_floor", "town",
      "I_index_50", "I_index_500", "I_index_1000", "I_index_5000", "I_index_10000", "II_index_50", "II_index_500",
      "II_index_1000", "II_index_5000", "II_index_10000", "III_index_50", "III_index_500", "III_index_1000",
      "III_index_5000", "III_index_10000", "IV_index_50", "IV_index_500", "IV_index_1000", "IV_index_5000",
      "IV_index_10000", "V_index_50", "V_index_500", "V_index_1000", "V_index_5000", "V_index_10000", "VI_index_50",
      "VI_index_500", "VI_index_1000", "VI_index_5000", "VI_index_10000", "VII_index_50", "VII_index_500",
      "VII_index_1000", "VII_index_5000", "VII_index_10000", "VIII_index_50", "VIII_index_500", "VIII_index_1000",
      "VIII_index_5000", "VIII_index_10000", "IX_index_50", "IX_index_500", "IX_index_1000", "IX_index_5000",
      "IX_index_10000", "X_index_50", "X_index_500", "X_index_1000", "X_index_5000", "X_index_10000", "XI_index_50",
      "XI_index_500", "XI_index_1000", "XI_index_5000", "XI_index_10000", "XII_index_50", "XII_index_500",
      "XII_index_1000", "XII_index_5000", "XII_index_10000", "XIII_index_50", "XIII_index_500", "XIII_index_1000",
      "XIII_index_5000", "XIII_index_10000", "XIV_index_50", "XIV_index_500", "XIV_index_1000", "XIV_index_5000",
      "XIV_index_10000"
]].astype('category')
data_t[[
      "building_id", "city", "building_type", "building_use", "parking_way", "total_floor", "town",
      "I_index_50", "I_index_500", "I_index_1000", "I_index_5000", "I_index_10000", "II_index_50", "II_index_500",
      "II_index_1000", "II_index_5000", "II_index_10000", "III_index_50", "III_index_500", "III_index_1000",
      "III_index_5000", "III_index_10000", "IV_index_50", "IV_index_500", "IV_index_1000", "IV_index_5000",
      "IV_index_10000", "V_index_50", "V_index_500", "V_index_1000", "V_index_5000", "V_index_10000", "VI_index_50",
      "VI_index_500", "VI_index_1000", "VI_index_5000", "VI_index_10000", "VII_index_50", "VII_index_500",
      "VII_index_1000", "VII_index_5000", "VII_index_10000", "VIII_index_50", "VIII_index_500", "VIII_index_1000",
      "VIII_index_5000", "VIII_index_10000", "IX_index_50", "IX_index_500", "IX_index_1000", "IX_index_5000",
      "IX_index_10000", "X_index_50", "X_index_500", "X_index_1000", "X_index_5000", "X_index_10000", "XI_index_50",
      "XI_index_500", "XI_index_1000", "XI_index_5000", "XI_index_10000", "XII_index_50", "XII_index_500",
      "XII_index_1000", "XII_index_5000", "XII_index_10000", "XIII_index_50", "XIII_index_500", "XIII_index_1000",
      "XIII_index_5000", "XIII_index_10000", "XIV_index_50", "XIV_index_500", "XIV_index_1000", "XIV_index_5000",
      "XIV_index_10000"
]] = data_t[[
      "building_id", "city", "building_type", "building_use", "parking_way", "total_floor", "town",
      "I_index_50", "I_index_500", "I_index_1000", "I_index_5000", "I_index_10000", "II_index_50", "II_index_500",
      "II_index_1000", "II_index_5000", "II_index_10000", "III_index_50", "III_index_500", "III_index_1000",
      "III_index_5000", "III_index_10000", "IV_index_50", "IV_index_500", "IV_index_1000", "IV_index_5000",
      "IV_index_10000", "V_index_50", "V_index_500", "V_index_1000", "V_index_5000", "V_index_10000", "VI_index_50",
      "VI_index_500", "VI_index_1000", "VI_index_5000", "VI_index_10000", "VII_index_50", "VII_index_500",
      "VII_index_1000", "VII_index_5000", "VII_index_10000", "VIII_index_50", "VIII_index_500", "VIII_index_1000",
      "VIII_index_5000", "VIII_index_10000", "IX_index_50", "IX_index_500", "IX_index_1000", "IX_index_5000",
      "IX_index_10000", "X_index_50", "X_index_500", "X_index_1000", "X_index_5000", "X_index_10000", "XI_index_50",
      "XI_index_500", "XI_index_1000", "XI_index_5000", "XI_index_10000", "XII_index_50", "XII_index_500",
      "XII_index_1000", "XII_index_5000", "XII_index_10000", "XIII_index_50", "XIII_index_500", "XIII_index_1000",
      "XIII_index_5000", "XIII_index_10000", "XIV_index_50", "XIV_index_500", "XIV_index_1000", "XIV_index_5000",
      "XIV_index_10000"
]].astype('category')

# 去掉不要變數
d1 = data.drop(['building_id', 'parking_area', 'parking_price', 'txn_floor', 'village', ], axis=1)  # axis=1代表col
d1_t = data_t.drop(['building_id', 'parking_area', 'parking_price', 'txn_floor', 'village', ], axis=1)

# 新增變數
d1["dt_ratio"] = (d1["txn_dt"] - d1["building_complete_dt"]) / d1["txn_dt"]
d1["bc_ratio"] = (d1["txn_dt"] - d1["building_complete_dt"]) / d1["building_complete_dt"]
d1["bo_de_ratio"] = d1["born_rate"] / d1["death_rate"]
d1["dt_bc_ratio"] = d1["txn_dt"] / d1["building_complete_dt"]
d1["txn_dt2"] = np.log(d1["txn_dt"])
d1["building_type2"] = d1["building_type"]
d1["bu_la_area"] = d1["building_area"] + d1["land_area"]
d1['building_age_jo'] = d1['txn_dt'] - d1['building_complete_dt']

d1_t["dt_ratio"] = (d1_t["txn_dt"] - d1_t["building_complete_dt"]) / d1_t["txn_dt"]
d1_t["bc_ratio"] = (d1_t["txn_dt"] - d1_t["building_complete_dt"]) / d1_t["building_complete_dt"]
d1_t["bo_de_ratio"] = d1_t["born_rate"] / d1_t["death_rate"]
d1_t["dt_bc_ratio"] = d1_t["txn_dt"] / d1_t["building_complete_dt"]
d1_t["txn_dt2"] = np.log(d1_t["txn_dt"])
d1_t["building_type2"] = d1_t["building_type"]
d1_t["bu_la_area"] = d1_t["building_area"] + d1_t["land_area"]
d1_t['building_age_jo'] = d1_t['txn_dt'] - d1_t['building_complete_dt']


# 指定變數
d2 = d1[[
      "total_price",
      "txn_dt", "building_complete_dt",
      "dt_ratio", "floor_class_jo", "village_income_median", "bo_de_ratio", "bc_ratio", "txn_dt2","bu_la_area",'building_age_jo',
      "land_area", "building_area", "lat", "lon", "town_population", "town_area",
      "town_population_density", "doc_rate", "master_rate", "bachelor_rate", "jobschool_rate", "highschool_rate",
      "junior_rate", "elementary_rate", "high_graduate_jo", "high/low_graduate_jo",
      "born_rate", "death_rate", "marriage_rate", "divorce_rate",
      "N_50", "N_500", "N_1000", "N_5000", "N_10000", "I_10", "I_50", "I_100", "I_250", "I_500", "I_1000", "I_5000",
      "I_10000", "I_MIN", "II_10", "II_50", "II_100", "II_250", "II_500", "II_1000", "II_5000", "II_10000", "II_MIN",
      "III_10", "III_50", "III_100", "III_250", "III_500", "III_1000", "III_5000", "III_10000", "III_MIN", "IV_10",
      "IV_50", "IV_100", "IV_250", "IV_500", "IV_1000", "IV_5000", "IV_10000", "IV_MIN", "V_10", "V_50", "V_100",
      "V_250", "V_500", "V_1000", "V_5000", "V_10000", "V_MIN", "VI_10", "VI_50", "VI_100", "VI_250", "VI_500",
      "VI_1000", "VI_5000", "VI_10000", "VI_MIN", "VII_10", "VII_50", "VII_100", "VII_250", "VII_500", "VII_1000",
      "VII_5000", "VII_10000", "VII_MIN", "VIII_10", "VIII_50", "VIII_100", "VIII_250", "VIII_500", "VIII_1000",
      "VIII_5000", "VIII_10000", "VIII_MIN", "IX_10", "IX_50", "IX_100", "IX_250", "IX_500", "IX_1000", "IX_5000",
      "IX_10000", "IX_MIN", "X_10", "X_50", "X_100", "X_250", "X_500", "X_1000", "X_5000", "X_10000", "X_MIN", "XI_10",
      "XI_50", "XI_100", "XI_250", "XI_500", "XI_1000", "XI_5000", "XI_10000", "XI_MIN", "XII_10", "XII_50", "XII_100",
      "XII_250", "XII_500", "XII_1000", "XII_5000", "XII_10000", "XII_MIN", "XIII_10", "XIII_50", "XIII_100",
      "XIII_250", "XIII_500", "XIII_1000", "XIII_5000", "XIII_10000", "XIII_MIN", "XIV_10", "XIV_50", "XIV_100",
      "XIV_250",
      "XIV_500", "XIV_1000", "XIV_5000", "XIV_10000", "XIV_MIN",

      "city", "building_type", "parking_way", "building_type2",
      "I_index_50", "I_index_500", "I_index_1000", "I_index_5000", "I_index_10000", "II_index_50", "II_index_500",
      "II_index_1000", "II_index_5000", "II_index_10000", "III_index_50", "III_index_500", "III_index_1000",
      "III_index_5000", "III_index_10000", "IV_index_50", "IV_index_500", "IV_index_1000", "IV_index_5000",
      "IV_index_10000", "V_index_50", "V_index_500", "V_index_1000", "V_index_5000", "V_index_10000", "VI_index_50",
      "VI_index_500", "VI_index_1000", "VI_index_5000", "VI_index_10000", "VII_index_50", "VII_index_500",
      "VII_index_1000", "VII_index_5000", "VII_index_10000", "VIII_index_50", "VIII_index_500", "VIII_index_1000",
      "VIII_index_5000", "VIII_index_10000", "IX_index_50", "IX_index_500", "IX_index_1000", "IX_index_5000",
      "IX_index_10000", "X_index_50", "X_index_500", "X_index_1000", "X_index_5000", "X_index_10000", "XI_index_50",
      "XI_index_500", "XI_index_1000", "XI_index_5000", "XII_index_500",
      "XII_index_1000", "XII_index_5000", "XII_index_10000", "XIII_index_50", "XIII_index_500", "XIII_index_1000",
      "XIII_index_5000", "XIII_index_10000", "XIV_index_50", "XIV_index_500", "XIV_index_1000", "XIV_index_5000",
      "XIV_index_10000"
]]
d2_t = d1_t[[
      "txn_dt", "building_complete_dt",
      "dt_ratio", "floor_class_jo", "village_income_median", "bo_de_ratio", "bc_ratio", "txn_dt2","bu_la_area",'building_age_jo',
      "land_area", "building_area", "lat", "lon", "town_population", "town_area",
      "town_population_density", "doc_rate", "master_rate", "bachelor_rate", "jobschool_rate", "highschool_rate",
      "junior_rate", "elementary_rate", "high_graduate_jo", "high/low_graduate_jo",
      "born_rate", "death_rate", "marriage_rate", "divorce_rate",
      "N_50", "N_500", "N_1000", "N_5000", "N_10000", "I_10", "I_50", "I_100", "I_250", "I_500", "I_1000", "I_5000",
      "I_10000", "I_MIN", "II_10", "II_50", "II_100", "II_250", "II_500", "II_1000", "II_5000", "II_10000", "II_MIN",
      "III_10", "III_50", "III_100", "III_250", "III_500", "III_1000", "III_5000", "III_10000", "III_MIN", "IV_10",
      "IV_50", "IV_100", "IV_250", "IV_500", "IV_1000", "IV_5000", "IV_10000", "IV_MIN", "V_10", "V_50", "V_100",
      "V_250", "V_500", "V_1000", "V_5000", "V_10000", "V_MIN", "VI_10", "VI_50", "VI_100", "VI_250", "VI_500",
      "VI_1000", "VI_5000", "VI_10000", "VI_MIN", "VII_10", "VII_50", "VII_100", "VII_250", "VII_500", "VII_1000",
      "VII_5000", "VII_10000", "VII_MIN", "VIII_10", "VIII_50", "VIII_100", "VIII_250", "VIII_500", "VIII_1000",
      "VIII_5000", "VIII_10000", "VIII_MIN", "IX_10", "IX_50", "IX_100", "IX_250", "IX_500", "IX_1000", "IX_5000",
      "IX_10000", "IX_MIN", "X_10", "X_50", "X_100", "X_250", "X_500", "X_1000", "X_5000", "X_10000", "X_MIN", "XI_10",
      "XI_50", "XI_100", "XI_250", "XI_500", "XI_1000", "XI_5000", "XI_10000", "XI_MIN", "XII_10", "XII_50", "XII_100",
      "XII_250", "XII_500", "XII_1000", "XII_5000", "XII_10000", "XII_MIN", "XIII_10", "XIII_50", "XIII_100",
      "XIII_250", "XIII_500", "XIII_1000", "XIII_5000", "XIII_10000", "XIII_MIN", "XIV_10", "XIV_50", "XIV_100",
      "XIV_250",
      "XIV_500", "XIV_1000", "XIV_5000", "XIV_10000", "XIV_MIN",

      "city", "building_type", "parking_way", "building_type2",
      "I_index_50", "I_index_500", "I_index_1000", "I_index_5000", "I_index_10000", "II_index_50", "II_index_500",
      "II_index_1000", "II_index_5000", "II_index_10000", "III_index_50", "III_index_500", "III_index_1000",
      "III_index_5000", "III_index_10000", "IV_index_50", "IV_index_500", "IV_index_1000", "IV_index_5000",
      "IV_index_10000", "V_index_50", "V_index_500", "V_index_1000", "V_index_5000", "V_index_10000", "VI_index_50",
      "VI_index_500", "VI_index_1000", "VI_index_5000", "VI_index_10000", "VII_index_50", "VII_index_500",
      "VII_index_1000", "VII_index_5000", "VII_index_10000", "VIII_index_50", "VIII_index_500", "VIII_index_1000",
      "VIII_index_5000", "VIII_index_10000", "IX_index_50", "IX_index_500", "IX_index_1000", "IX_index_5000",
      "IX_index_10000", "X_index_50", "X_index_500", "X_index_1000", "X_index_5000", "X_index_10000", "XI_index_50",
      "XI_index_500", "XI_index_1000", "XI_index_5000", "XII_index_500",
      "XII_index_1000", "XII_index_5000", "XII_index_10000", "XIII_index_50", "XIII_index_500", "XIII_index_1000",
      "XIII_index_5000", "XIII_index_10000", "XIV_index_50", "XIV_index_500", "XIV_index_1000", "XIV_index_5000",
      "XIV_index_10000"
]]

# one-hot encoding
d2 = pd.get_dummies(d2)
d2_t = pd.get_dummies(d2_t)


In [ ]:
# txn_dt 與 total_price線性回歸看離群值
sns.regplot(x=d2['txn_dt'],y=np.log(d2["total_price"]/d2['txn_dt']))

In [ ]:
# bu_la_area 與 total_price線性回歸看離群值
sns.regplot(x=d2["bu_la_area"],y=np.log(d2["total_price"]/d2["bu_la_area"]))

In [ ]:
d2 = d2[d2["total_price"]/d2["bu_la_area"]/100000 < 100]
len(d2)

In [ ]:
# building_age_jo 與 total_price線性回歸看離群值
sns.regplot(x=d2['building_age_jo'],y=np.log(d2["total_price"]/d2['building_age_jo']))

In [ ]:
d2 = d2[d2["total_price"]/d2['building_age_jo']/1000000 < 15]
len(d2)

In [ ]:
# building_area 與 total_price線性回歸看離群值
sns.regplot(x=d2["building_area"],y=np.log(d2["total_price"]/d2["building_area"]))

In [ ]:
d2 = d2[d2["total_price"]/d2["building_area"]/10000000 < 10]
len(d2)

In [ ]:
# XII_MIN 與 total_price線性回歸看離群值
sns.regplot(x=d2["XII_MIN"],y=np.log(d2["total_price"]/d2["XII_MIN"]))

In [ ]:
d2 = d2[d2["total_price"]/d2["XII_MIN"]/100000000 < 15]
len(d2)

In [ ]:
# VII_MIN 與 total_price線性回歸看離群值
sns.regplot(x=d2["VII_MIN"],y=np.log(d2["total_price"]/d2["VII_MIN"]))

In [ ]:
d2 = d2[d2["total_price"]/d2["VII_MIN"]/100000000 < 10]
len(d2)

In [ ]:
# XIV_MIN 與 total_price線性回歸看離群值
sns.regplot(x=d2["XIV_MIN"],y=np.log(d2["total_price"]/d2["XIV_MIN"]))

In [ ]:
d2 = d2[d2["total_price"]/d2["XIV_MIN"]/100000000 < 3]
len(d2)

In [ ]:
# VIII_MIN 與 total_price線性回歸看離群值
sns.regplot(x=d2["VIII_MIN"],y=np.log(d2["total_price"]/d2["VIII_MIN"]))

In [ ]:
d2 = d2[d2["total_price"]/d2["VIII_MIN"]/100000000 < 8]
len(d2)

In [ ]:
# land_area 與 total_price線性回歸看離群值
sns.regplot(x=d2["land_area"],y=np.log(d2["total_price"]/d2["land_area"]))

In [ ]:
d2 = d2[d2["total_price"]/d2["land_area"]/10000000 < 2.5]
len(d2)

In [ ]:
y = np.log(d2.total_price)
x = d2.drop('total_price', axis=1)
train_x, test_x, train_y, test_y = train_test_split(x.as_matrix(), y.as_matrix(), test_size=0.005, random_state=0)  # 切資料,轉成矩陣
test_data_x = d2_t.as_matrix()
my_imputer = Imputer()
train_x = my_imputer.fit_transform(train_x)
clf = xgb.XGBRegressor(
    colsample_bylevel=0.7,
    colsample_bytree = 0.75,
    max_depth = 7, 
    gamma=0,
    min_child_weight=4,    
    learning_rate = 0.033,
    max_delta_step=0,
    missing=None,
    n_estimators = 30000,
    nthread= 12,
    silent = True,
    objective = 'reg:gamma', #'reg:linear', 'reg:linear' 
    reg_lambda = 0.8,
    scale_pos_weiht = 1,
    seed = 1234,
    subsample = 1,
    default='gpu_predictor',
    tree_method = 'gpu_hist',
    gpu_id = 0,
    max_bin = 256
    )
clf.fit(train_x, train_y, verbose=False)
predictions = clf.predict(test_x)
AVM(np.exp(test_y),np.exp(predictions))

In [ ]:
fig,ax = plt.subplots(figsize=(15,15))
plot_importance(clf,
                height=0.5,
                ax=ax,
                max_num_features=64)
plt.show()

In [ ]:
test_data_x = d2_t.as_matrix()
test_predictions = clf.predict(test_data_x)
predict_price = {"building_id":data_t["building_id"],"total_price":np.exp(test_predictions)}
df_test = DataFrame(predict_price)
df_test.to_csv("submit_test_me_outly_0.005_4.csv",encoding='utf-8',index=False)